In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from preprocessing import *

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('data/보유리스크율_일반')
BASE_YYMM = '201912'
BASE_YYYY = BASE_YYMM[:4]

In [3]:
# 데이터 불러오기
일반_원수_직전3년연간경과보험료 = pd.read_excel(FILE_PATH / f'일반_원수_직전3년연간경과보험료_{202012}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'OGL_ELP_PRM': float})
일반_출재_직전3년연간경과보험료 = pd.read_excel(FILE_PATH / f'일반_출재_직전3년연간경과보험료_{202012}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RN_ELP_PRM': float})
일반_원수_직전3년연간손해액 = pd.read_excel(FILE_PATH / f'일반_원수_직전3년연간손해액_{202012}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'PDC_CD': str, 'OGL_LOSS': float})
일반_출재_직전3년연간손해액 = pd.read_excel(FILE_PATH / f'일반_출재_직전3년연간손해액_{202012}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'PDC_CD': str, 'RN_LOSS': float})
일반_특약보종별_직전1년경과보험료 = pd.read_excel(FILE_PATH / f'일반_특약보종별_직전1년경과보험료_{BASE_YYMM}.xlsx',
    dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RRNR_TTY_CD': str, 'ELP_PRM': float, 'RN_ELP_PRM': float}) \
    .rename(columns={'ELP_PRM': 'OGL_ELP_PRM'}) # TODO: ELP_PRM -> OGL_ELP_PRM로 수정하기
일반_특약보종별_직전1년손해액 = pd.read_excel(FILE_PATH / f'일반_특약보종별_직전1년손해액_{BASE_YYMM}.xlsx',
    dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RRNR_TTY_CD': str, 'OGL_LOSS': float, 'RN_LOSS': float})
일반_특약수수료 = pd.read_excel(FILE_PATH / f'일반_특약수수료_{202012}.xlsx', dtype={'RRNR_TTY_CD': str, 'TTY_YR': str})
일반_특약정보 = pd.read_excel(FILE_PATH / f'일반_특약정보_{BASE_YYMM}.xlsx', dtype={'RRNR_TTY_CD': str, 'TTY_YR': str})
일반_상품정보 = pd.read_excel(FILE_PATH / '일반_상품정보.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})
산업_손해율_변동계수 = pd.read_excel(FILE_PATH / '산업_손해율_변동계수.xlsx')

In [4]:
pd.options.display.float_format = '{:,.3f}'.format

# 특약정보
## 근재보험특약, 기술보험특약, 재물보험특약, 배상책임보험특약, 해외PST
일반_특약정보_가공 = 일반_특약정보.copy()
일반_특약정보_가공['TTY_CD_GRP'] = None
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("(재물보험|패키지)"), 'TTY_CD_GRP'] = '재물보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("기술보험"), 'TTY_CD_GRP'] = '기술보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("근재보험"), 'TTY_CD_GRP'] = '근재보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("배상책임보험"), 'TTY_CD_GRP'] = '배상책임보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("해외PST"), 'TTY_CD_GRP'] = '해외PST'

# 특약수수료
일반_특약수수료_가공 = 일반_특약수수료.copy()
일반_특약수수료_가공 = 일반_특약수수료_가공.merge(일반_특약정보_가공, on=['RRNR_TTY_CD', 'TTY_YR'], how='left').query('~TTY_CD_GRP.isna()')
일반_특약수수료_가공 = 일반_특약수수료_가공[['TTY_CD_GRP', 'TTY_YR', 'CMSN_ADD_RT', 'CMSN_MULT_RT', 'BSE_LSRT', 'PVSN_CMSN_RT', 'LWT_CMSN_RT', 'TOP_CMSN_RT']].drop_duplicates()

# 직전3년평균손해율
일반_원수_직전3년연간경과보험료_가공 = 일반_원수_직전3년연간경과보험료.copy()
일반_원수_직전3년연간경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_직전3년연간경과보험료_가공)
일반_원수_직전3년연간경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_직전3년연간경과보험료_가공)
일반_원수_직전3년연간경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_직전3년연간경과보험료_가공, 일반_상품정보)
일반_원수_직전3년연간경과보험료_집계 = 일반_원수_직전3년연간경과보험료_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['OGL_ELP_PRM']].sum()

일반_출재_직전3년연간경과보험료_가공 = 일반_출재_직전3년연간경과보험료.copy()
일반_출재_직전3년연간경과보험료_가공['RRNR_DVCD'] = '03'
일반_출재_직전3년연간경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_직전3년연간경과보험료_가공)
일반_출재_직전3년연간경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_직전3년연간경과보험료_가공, 일반_상품정보)
일반_출재_직전3년연간경과보험료_집계 = 일반_출재_직전3년연간경과보험료_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['RN_ELP_PRM']].sum()

일반_원수_직전3년연간손해액_가공 = 일반_원수_직전3년연간손해액.copy()
일반_원수_직전3년연간손해액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_직전3년연간손해액_가공)
일반_원수_직전3년연간손해액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_직전3년연간손해액_가공)
일반_원수_직전3년연간손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_직전3년연간손해액_가공, 일반_상품정보)
일반_원수_직전3년연간손해액_집계 = 일반_원수_직전3년연간손해액_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['OGL_LOSS']].sum()

일반_출재_직전3년연간손해액_가공 = 일반_출재_직전3년연간손해액.copy()
일반_출재_직전3년연간손해액_가공['RRNR_DVCD'] = '03'
일반_출재_직전3년연간손해액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_직전3년연간손해액_가공)
일반_출재_직전3년연간손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_직전3년연간손해액_가공, 일반_상품정보)
일반_출재_직전3년연간손해액_집계 = 일반_출재_직전3년연간손해액_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['RN_LOSS']].sum()

일반_직전3년평균손해율 = 일반_원수_직전3년연간경과보험료_집계 \
    .merge(일반_출재_직전3년연간경과보험료_집계, on=['FY', 'BOZ_CD'], how='outer') \
    .merge(일반_원수_직전3년연간손해액_집계, on=['FY', 'BOZ_CD'], how='outer') \
    .merge(일반_출재_직전3년연간손해액_집계, on=['FY', 'BOZ_CD'], how='outer')
일반_직전3년평균손해율[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']] = 일반_직전3년평균손해율[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']].fillna(0)
일반_직전3년평균손해율.eval('RET_ELP_PRM = OGL_ELP_PRM - RN_ELP_PRM', inplace=True)
일반_직전3년평균손해율.eval('RET_LOSS = OGL_LOSS - RN_LOSS', inplace=True)
일반_직전3년평균손해율.eval('LOSS_RATIO = RET_LOSS/RET_ELP_PRM', inplace=True)
일반_직전3년평균손해율 = 일반_직전3년평균손해율.groupby('BOZ_CD', as_index=False)['LOSS_RATIO'].mean()

# 특약보종별 직전1년경과보험료
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료.copy()
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료_가공.query('~RRNR_DAT_DVCD.isna()').reset_index(drop=True)
일반_특약보종별_직전1년경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_특약보종별_직전1년경과보험료_가공)
일반_특약보종별_직전1년경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_특약보종별_직전1년경과보험료_가공)
일반_특약보종별_직전1년경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_특약보종별_직전1년경과보험료_가공, 일반_상품정보)
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료_가공.merge(일반_특약정보_가공, on='RRNR_TTY_CD', how='left')
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료_가공.groupby(['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], as_index=False)[['OGL_ELP_PRM', 'RN_ELP_PRM']].sum()

# 특약보종별 직전1년손해액
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액.copy()
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액_가공.query('~RRNR_DAT_DVCD.isna()').reset_index(drop=True)
일반_특약보종별_직전1년손해액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_특약보종별_직전1년손해액_가공)
일반_특약보종별_직전1년손해액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_특약보종별_직전1년손해액_가공)
일반_특약보종별_직전1년손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_특약보종별_직전1년손해액_가공, 일반_상품정보)
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액_가공.merge(일반_특약정보_가공, on='RRNR_TTY_CD', how='left')
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액_가공.groupby(['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], as_index=False)[['OGL_LOSS', 'RN_LOSS']].sum()

일반_특약보종별_실적 = 일반_특약보종별_직전1년경과보험료_가공 \
    .merge(일반_특약보종별_직전1년손해액_가공, on=['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], how='outer')
일반_특약보종별_실적[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']] = 일반_특약보종별_실적[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']].fillna(0)

# 특약보종별 실적 집계
# 일반_특약보종별_실적_가공 = []
# for tty_cd_grp in ['재물보험특약', '기술보험특약', '근재보험특약', '배상책임보험특약', '해외PST']:
#     STRT_YYYY = str(int(BASE_YYYY)  - (7 if tty_cd_grp in ['근재보험특약', '배상책임보험특약'] else 5))
#     일반_특약보종별_실적_가공.append(일반_특약보종별_실적.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR > @STRT_YYYY'))
# 일반_특약보종별_실적_가공 = pd.concat(일반_특약보종별_실적_가공, axis=0).reset_index(drop=True)
일반_특약보종별_실적_가공 = []
for boz_cd in ['A001', 'A002', 'A003', 'A004', 'A005', 'A006', 'A007', 'A008', 'A009', 'A010', 'A011']:
    STRT_YYYY = str(int(BASE_YYYY)  - (7 if boz_cd in ['A005', 'A006'] else 5))
    일반_특약보종별_실적_가공.append(일반_특약보종별_실적.query('BOZ_CD == @boz_cd').query('TTY_YR > @STRT_YYYY'))
일반_특약보종별_실적_가공 = pd.concat(일반_특약보종별_실적_가공, axis=0).reset_index(drop=True)

C:\Users\HOME-Notebook\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [19]:
pd.options.display.float_format = '{:,.0f}'.format

# 보유리스크율
## 위험계수적용법
일반_보유리스크율_위험계수적용법 = []
for tty_cd_grp in ['근재보험특약', '기술보험특약', '재물보험특약', '배상책임보험특약', '해외PST']:
    for boz_cd in ['A001', 'A002', 'A003', 'A004', 'A005', 'A006', 'A007', 'A008', 'A009', 'A010', 'A011']:
        prem = 일반_특약보종별_실적_가공.query('TTY_CD_GRP == @tty_cd_grp').query('BOZ_CD == @boz_cd').groupby('TTY_YR', as_index=False)[['OGL_ELP_PRM', 'RN_ELP_PRM']].sum()
        if len(prem) == 0: continue
        comm = 일반_특약수수료_가공.query('TTY_CD_GRP == @tty_cd_grp').drop('TTY_CD_GRP', axis=1)
        loss_ratio = 일반_직전3년평균손해율.query('BOZ_CD == @boz_cd')['LOSS_RATIO'].values[0]
        risk_coef = 0.55 # TODO: boz_cd별 적용위험계수로 수정
        일반_보유리스크율_위험계수적용법_특약보종별 = get_ret_risk_rate_by_risk_coef(comm, prem, loss_ratio, risk_coef)
        일반_보유리스크율_위험계수적용법_특약보종별.insert(0, 'BOZ_CD', boz_cd)
        일반_보유리스크율_위험계수적용법_특약보종별.insert(0, 'TTY_CD_GRP', tty_cd_grp)
        일반_보유리스크율_위험계수적용법.append(일반_보유리스크율_위험계수적용법_특약보종별)
일반_보유리스크율_위험계수적용법 = pd.concat(일반_보유리스크율_위험계수적용법, axis=0)

In [20]:
일반_보유리스크율_위험계수적용법

,TTY_CD_GRP,BOZ_CD,TTY_YR,OGL_ELP_PRM,RN_ELP_PRM,OGL_EXP_LOSS_BASE,OGL_EXP_LOSS_SHOCKED,OGL_EXP_LOSS_DIFF,RET_EXP_LOSS_BASE,RET_EXP_LOSS_SHOCKED,RET_EXP_LOSS_DIFF,COMM_BASE,COMM_SHOCKED,COMM_DIFF
0,근재보험특약,A005,2013,"159,604","28,817","138,592","247,386","108,794","113,569","202,720","89,151","5,686","5,187",-499
1,근재보험특약,A005,2014,"16,407,076","3,275,285","14,247,100","25,430,968","11,183,868","11,403,003","20,354,276","8,951,273","646,277","589,551","-56,726"
2,근재보험특약,A005,2015,"14,757,334","2,756,334","12,814,545","22,873,868","10,059,323","10,421,079","18,601,550","8,180,471","543,878","496,140","-47,738"
3,근재보험특약,A005,2016,"1,134,705,058","226,351,667","985,322,189","1,758,792,840","773,470,651","788,769,509","1,407,947,756","619,178,247","44,663,543","40,743,300","-3,920,243"
4,근재보험특약,A005,2017,"84,867,675","15,279,460","73,694,924","131,544,896","57,849,972","60,426,991","107,861,733","47,434,742","3,014,932","2,750,303","-264,629"
5,근재보험특약,A005,2018,"6,018,237,352","1,190,887,076","5,225,941,982","9,328,267,896","4,102,325,913","4,191,834,086","7,482,392,928","3,290,558,842","234,984,955","214,359,674","-20,625,281"
6,근재보험특약,A005,2019,"8,596,002,347","1,700,595,263","7,464,346,605","13,323,803,638","5,859,457,033","5,987,633,132","10,687,880,980","4,700,247,848","335,560,196","306,107,147","-29,453,049"
0,기술보험특약,A002,2015,"37,594,020","15,393,040","13,770,368","58,270,731","44,500,363","8,132,029","34,411,519","26,279,490","8,357,910","4,925,773","-3,432,137"
1,기술보험특약,A002,2015,"37,594,020","15,393,040","13,770,368","58,270,731","44,500,363","8,132,029","34,411,519","26,279,490","8,357,910","4,925,773","-3,432,137"
2,기술보험특약,A002,2016,"207,109,805","50,539,226","75,862,551","321,020,198","245,157,647","57,350,464","242,684,397","185,333,933","27,340,045","16,172,552","-11,167,493"


In [ ]:
def get_ret_risk_rate_by_loss_dist(comm: pd.DataFrame, prem: pd.DataFrame, loss: pd.DataFrame, loss_ratio: float, cv: pd.DataFrame) -> pd.DataFrame:
    """특약별 보유리스크율(손해율분포법)"""